import libraries

In [2]:
import os
import json
import numpy as np
import pandas as pd
from utils import load_json

Helper Functions

In [3]:
def load_json(path):

    with open(path, "r") as f:

        return json.load(f)

Consolidate results

In [4]:
# Manually add in (hack)

all_folders = []
all_test_performances = {"loss": {}, "jaccard": {}} 

for folder in ["./models_cached/", "../mist/models_cached/"]:

    for model_type in os.listdir(folder):
        subfolder = os.path.join(folder, model_type)
        for dataset in os.listdir(subfolder):
            subsubfolder = os.path.join(subfolder, dataset)
            for checkpoint in os.listdir(subsubfolder):
                all_folders.append(os.path.join(subsubfolder, checkpoint))

for f in all_folders:

    try:
        expt_name = f.split("/")[-1]
        test_performance = load_json(os.path.join(f, "test_performance.json"))
        for k, v in test_performance.items():
            all_test_performances[k][expt_name] = round(v, 5)
    except Exception as e:
        print(e)
        continue

FileNotFoundError: [Errno 2] No such file or directory: './models_cached/'

Format the table

In [ ]:
results = pd.DataFrame.from_dict(all_test_performances)
results = results.reset_index()
results = results.rename(columns = {"index": "expt_name"})
results.loc[:, "dataset"] = results.loc[:, "expt_name"].apply(lambda x: x.split("_")[0]) 
results.loc[:, "meta"] = results.loc[:, "expt_name"].apply(lambda x: "meta" in x) 
results.loc[:, "model"] = results.loc[:, "expt_name"].apply(lambda x: x.split("_")[1]) 
results.loc[:, "split"] = results.loc[:, "expt_name"].apply(lambda x: "_".join(x.split("_")[-2:]).replace("4096_", "")) 

results = results.loc[:, ["dataset", "model", "split", "meta", "loss", "jaccard"]]

results = results.sort_values(by = 'split', ascending = False)
results = results.sort_values(by = 'model', ascending = False)
results = results.sort_values(by = 'dataset', ascending = False)

results

,dataset,model,split,meta,loss,jaccard
49,NIST2023,formula,inchikey_vanilla,False,0.05918,0.24013
43,NIST2023,binned,random,False,0.07459,0.21556
16,NIST2023,binned,random,True,0.07128,0.21649
18,NIST2023,binned,inchikey_vanilla,True,0.07259,0.20871
44,NIST2023,binned,inchikey_vanilla,False,0.07031,0.20467
...,...,...,...,...,...,...
33,C,formula,inchikey_vanilla,False,0.08101,0.21179
9,C,MS,random,True,0.05212,0.34188
34,C,MS,inchikey_vanilla,False,0.08187,0.21414
10,C,formula,inchikey_vanilla,True,0.08182,0.21629


In [ ]:
C = results.loc[results.loc[:, "dataset"].apply(lambda x : "C" in x), :]
C.loc[C.loc[:, "meta"] == False, :]

,dataset,model,split,meta,loss,jaccard
55,C,MIST,inchikey_vanilla,False,0.05491,0.31058
53,C,MIST,scaffold_vanilla,False,0.07264,0.23677
38,C,formula,scaffold_vanilla,False,0.08817,0.18960
54,C,MIST,random,False,0.02919,0.54347
35,C,binned,scaffold_vanilla,False,0.09766,0.18688
39,C,MS,random,False,0.05475,0.33293
36,C,formula,random,False,0.05815,0.32369
41,C,binned,inchikey_vanilla,False,0.08352,0.21781
37,C,MS,scaffold_vanilla,False,0.08525,0.19624
33,C,formula,inchikey_vanilla,False,0.08101,0.21179


In [ ]:
MSG = results.loc[results.loc[:, "dataset"].apply(lambda x : "MSG" in x), :]
MSG.loc[MSG.loc[:, "meta"] == False, :]

,dataset,model,split,meta,loss,jaccard
25,MSG,MS,scaffold_vanilla,False,0.08595,0.19370
56,MSG,MIST,inchikey_vanilla,False,0.05815,0.29999
58,MSG,MIST,scaffold_vanilla,False,0.06512,0.26601
57,MSG,MIST,random,False,0.02223,0.67438
31,MSG,formula,random,False,0.03721,0.48166
28,MSG,MS,random,False,0.05189,0.37819
27,MSG,formula,inchikey_vanilla,False,0.08572,0.19580
32,MSG,formula,scaffold_vanilla,False,0.08465,0.19273
26,MSG,binned,random,False,0.05208,0.39982
30,MSG,MS,inchikey_vanilla,False,0.08459,0.19320


In [ ]:
NIST2023 = results.loc[results.loc[:, "dataset"].apply(lambda x : "NIST2023" in x), :]
NIST2023.loc[NIST2023.loc[:, "meta"] == False, :]

,dataset,model,split,meta,loss,jaccard
49,NIST2023,formula,inchikey_vanilla,False,0.05918,0.24013
43,NIST2023,binned,random,False,0.07459,0.21556
44,NIST2023,binned,inchikey_vanilla,False,0.07031,0.20467
48,NIST2023,MS,inchikey_vanilla,False,0.07274,0.20622
42,NIST2023,MS,scaffold_vanilla,False,0.07358,0.19934
45,NIST2023,MS,random,False,0.06858,0.21663
50,NIST2023,MIST,scaffold_vanilla,False,0.06124,0.25526
51,NIST2023,MIST,random,False,0.01628,0.62105
52,NIST2023,MIST,inchikey_vanilla,False,0.04789,0.30135
47,NIST2023,binned,scaffold_vanilla,False,0.07559,0.19980
